## Setup and Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols, gls
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings

warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('../data/processed/combined_nps_revenue.csv')
df['date_nps'] = pd.to_datetime(df['date_nps'])
df['year_month'] = pd.to_datetime(df['year_month'].astype(str))

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Create dummy variables for clinic
clinic_dummies = pd.get_dummies(df['clinic_id'], prefix='clinic', drop_first=True)
df = pd.concat([df, clinic_dummies], axis=1)

print('Data loaded for regression analysis:')
print(f'Shape: {df.shape}')
print(f'Missing values:\n{df[["nps_mean", "revenue_total", "nps_lag1"]].isnull().sum()}')

## Model 1: Simple Linear Regression

In [ ]:
# Simple regression: Revenue ~ NPS
df_simple = df[['nps_mean', 'revenue_total']].dropna()

X = sm.add_constant(df_simple[['nps_mean']])
y = df_simple['revenue_total']

model_simple = sm.OLS(y, X).fit()

print('='*80)
print('MODEL 1: SIMPLE LINEAR REGRESSION')
print('Specification: Revenue = β₀ + β₁(NPS) + ε')
print('='*80)
print(model_simple.summary())

# Save results
model1_results = {
    'Model': 'Simple Linear',
    'R-squared': model_simple.rsquared,
    'Adj R-squared': model_simple.rsquared_adj,
    'F-statistic': model_simple.fvalue,
    'Prob (F-stat)': model_simple.f_pvalue,
    'AIC': model_simple.aic,
    'BIC': model_simple.bic
}

## Diagnostic Plots: Model 1

In [ ]:
# Diagnostic plots
fig = plt.figure(figsize=(14, 10))

# Residuals vs Fitted
ax1 = plt.subplot(2, 2, 1)
residuals = model_simple.resid
fitted = model_simple.fittedvalues
ax1.scatter(fitted, residuals, alpha=0.5)
ax1.axhline(y=0, color='r', linestyle='--')
ax1.set_xlabel('Fitted Values')
ax1.set_ylabel('Residuals')
ax1.set_title('Residuals vs Fitted')
ax1.grid(True, alpha=0.3)

# Q-Q Plot
ax2 = plt.subplot(2, 2, 2)
stats.probplot(residuals, dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot')
ax2.grid(True, alpha=0.3)

# Histogram of Residuals
ax3 = plt.subplot(2, 2, 3)
ax3.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
ax3.axvline(x=0, color='r', linestyle='--')
ax3.set_xlabel('Residuals')
ax3.set_ylabel('Frequency')
ax3.set_title('Distribution of Residuals')
ax3.grid(True, alpha=0.3, axis='y')

# Scale-Location Plot
ax4 = plt.subplot(2, 2, 4)
standardized_resid = residuals / residuals.std()
ax4.scatter(fitted, np.sqrt(np.abs(standardized_resid)), alpha=0.5)
ax4.set_xlabel('Fitted Values')
ax4.set_ylabel('√|Std Residuals|')
ax4.set_title('Scale-Location Plot')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/reg_model1_diagnostics.png', dpi=300, bbox_inches='tight')
plt.show()

## Model 2: Multiple Regression with Derived Metrics

In [ ]:
# Multiple regression: Revenue ~ NPS + NPS_std + NPS_lag + Responses
model_cols = ['nps_mean', 'nps_median', 'nps_std', 'nps_responses']
df_multi = df[model_cols + ['revenue_total']].dropna()

X_multi = sm.add_constant(df_multi[model_cols])
y_multi = df_multi['revenue_total']

model_multi = sm.OLS(y_multi, X_multi).fit()

print('\n' + '='*80)
print('MODEL 2: MULTIPLE REGRESSION')
print('Specification: Revenue = β₀ + β₁(NPS_mean) + β₂(NPS_median) + β₃(NPS_std) + β₄(Responses) + ε')
print('='*80)
print(model_multi.summary())

model2_results = {
    'Model': 'Multiple Regression',
    'R-squared': model_multi.rsquared,
    'Adj R-squared': model_multi.rsquared_adj,
    'F-statistic': model_multi.fvalue,
    'Prob (F-stat)': model_multi.f_pvalue,
    'AIC': model_multi.aic,
    'BIC': model_multi.bic
}

## Model 3: Fixed Effects (Panel) Model

In [ ]:
# Fixed effects model: account for clinic-level heterogeneity
# Use clinic dummies
clinic_cols = [col for col in df.columns if col.startswith('clinic_')]
fe_cols = ['nps_mean'] + clinic_cols

df_fe = df[fe_cols + ['revenue_total']].dropna()

X_fe = sm.add_constant(df_fe[fe_cols])
y_fe = df_fe['revenue_total']

model_fe = sm.OLS(y_fe, X_fe).fit()

print('\n' + '='*80)
print('MODEL 3: FIXED EFFECTS (PANEL) MODEL')
print('Specification: Revenue = β₀ + β₁(NPS) + α₁(clinic_1) + ... + αₙ(clinic_n) + ε')
print('='*80)

# Display only NPS coefficient (clinic dummies too numerous)
print(f'\nKey Result - NPS Coefficient:')
print(f'  Coefficient: {model_fe.params["nps_mean"]:.4f}')
print(f'  Std Error: {model_fe.bse["nps_mean"]:.4f}')
print(f'  t-statistic: {model_fe.tvalues["nps_mean"]:.4f}')
print(f'  p-value: {model_fe.pvalues["nps_mean"]:.6f}')
print(f'  95% CI: [{model_fe.conf_int().loc["nps_mean", 0]:.4f}, {model_fe.conf_int().loc["nps_mean", 1]:.4f}]')

print(f'\nModel Fit Statistics:')
print(f'  R-squared: {model_fe.rsquared:.4f}')
print(f'  Adj R-squared: {model_fe.rsquared_adj:.4f}')
print(f'  F-statistic: {model_fe.fvalue:.4f}')
print(f'  Prob (F-stat): {model_fe.f_pvalue:.6f}')

model3_results = {
    'Model': 'Fixed Effects',
    'R-squared': model_fe.rsquared,
    'Adj R-squared': model_fe.rsquared_adj,
    'F-statistic': model_fe.fvalue,
    'Prob (F-stat)': model_fe.f_pvalue,
    'AIC': model_fe.aic,
    'BIC': model_fe.bic
}

## Model 4: Lagged Regression Model

In [ ]:
# Lagged model: Current revenue ~ Previous NPS + Current NPS
df_lag = df[['nps_mean', 'nps_lag1', 'revenue_total']].dropna()

X_lag = sm.add_constant(df_lag[['nps_mean', 'nps_lag1']])
y_lag = df_lag['revenue_total']

model_lag = sm.OLS(y_lag, X_lag).fit()

print('\n' + '='*80)
print('MODEL 4: LAGGED REGRESSION')
print('Specification: Revenue_t = β₀ + β₁(NPS_t) + β₂(NPS_t-1) + ε')
print('='*80)
print(model_lag.summary())

model4_results = {
    'Model': 'Lagged',
    'R-squared': model_lag.rsquared,
    'Adj R-squared': model_lag.rsquared_adj,
    'F-statistic': model_lag.fvalue,
    'Prob (F-stat)': model_lag.f_pvalue,
    'AIC': model_lag.aic,
    'BIC': model_lag.bic
}

## Model Comparison

In [ ]:
# Compare all models
model_comparison = pd.DataFrame([
    model1_results,
    model2_results,
    model3_results,
    model4_results
])

print('\n' + '='*80)
print('MODEL COMPARISON')
print('='*80)
print(model_comparison.to_string(index=False))

# Save comparison
model_comparison.to_csv('../data/processed/reg_model_comparison.csv', index=False)

# Identify best model by AIC
best_model_idx = model_comparison['AIC'].idxmin()
best_model_name = model_comparison.loc[best_model_idx, 'Model']
print(f'\nBest model by AIC: {best_model_name}')

## Visualization: Coefficient Estimates with Confidence Intervals

In [ ]:
# Extract coefficients and CIs for visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Model 1
ax = axes[0, 0]
coef1 = model_simple.params.drop('const')
ci1 = model_simple.conf_int().drop('const')
ax.errorbar(coef1.values, coef1.index, xerr=[coef1.values - ci1[0].values, ci1[1].values - coef1.values], fmt='o', markersize=8)
ax.axvline(x=0, color='r', linestyle='--', alpha=0.5)
ax.set_title('Model 1: Simple Linear', fontweight='bold')
ax.grid(True, alpha=0.3)

# Model 2
ax = axes[0, 1]
coef2 = model_multi.params.drop('const')
ci2 = model_multi.conf_int().drop('const')
ax.errorbar(coef2.values, coef2.index, xerr=[coef2.values - ci2[0].values, ci2[1].values - coef2.values], fmt='o', markersize=8)
ax.axvline(x=0, color='r', linestyle='--', alpha=0.5)
ax.set_title('Model 2: Multiple Regression', fontweight='bold')
ax.grid(True, alpha=0.3)

# Model 3 (FE)
ax = axes[1, 0]
coef3 = model_fe.params[['nps_mean']]
ci3 = model_fe.conf_int().loc[['nps_mean']]
ax.errorbar(coef3.values, ['NPS'], xerr=[coef3.values - ci3[0].values, ci3[1].values - coef3.values], fmt='o', markersize=8)
ax.axvline(x=0, color='r', linestyle='--', alpha=0.5)
ax.set_title('Model 3: Fixed Effects', fontweight='bold')
ax.grid(True, alpha=0.3)

# Model 4
ax = axes[1, 1]
coef4 = model_lag.params.drop('const')
ci4 = model_lag.conf_int().drop('const')
ax.errorbar(coef4.values, coef4.index, xerr=[coef4.values - ci4[0].values, ci4[1].values - coef4.values], fmt='o', markersize=8)
ax.axvline(x=0, color='r', linestyle='--', alpha=0.5)
ax.set_title('Model 4: Lagged Regression', fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/reg_coefficients_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## Residual Analysis and Assumptions Testing

In [ ]:
# Test for heteroscedasticity (Breusch-Pagan test)
from statsmodels.stats.diagnostic import het_breuschpagan

print('\n' + '='*80)
print('ASSUMPTION TESTS')
print('='*80)

# Test residuals from Model 1
residuals_1 = model_simple.resid
bp_test = het_breuschpagan(residuals_1, model_simple.model.exog)

print(f'\n1. HETEROSCEDASTICITY TEST (Breusch-Pagan)')
print(f'   H₀: Homoscedasticity (constant variance)')
print(f'   LM Statistic: {bp_test[0]:.4f}')
print(f'   p-value: {bp_test[1]:.6f}')
print(f'   Result: {"Reject H₀ - Heteroscedasticity detected" if bp_test[1] < 0.05 else "Fail to reject H₀ - Homoscedasticity assumed"}')

# Normality test (Jarque-Bera)
jb_test = stats.jarque_bera(residuals_1)

print(f'\n2. NORMALITY TEST (Jarque-Bera)')
print(f'   H₀: Residuals are normally distributed')
print(f'   JB Statistic: {jb_test[0]:.4f}')
print(f'   p-value: {jb_test[1]:.6f}')
print(f'   Result: {"Reject H₀ - Non-normal residuals" if jb_test[1] < 0.05 else "Fail to reject H₀ - Normality assumed"}')

# Durbin-Watson (autocorrelation)
dw_stat = sm.stats.durbin_watson(residuals_1)

print(f'\n3. AUTOCORRELATION TEST (Durbin-Watson)')
print(f'   DW Statistic: {dw_stat:.4f}')
print(f'   Interpretation: {"Positive autocorrelation" if dw_stat < 1.5 else "No autocorrelation" if dw_stat < 2.5 else "Negative autocorrelation"}')
print(f'   (Range: 0-4, where 2 = no autocorrelation)')

## Robustness Check: Outlier Sensitivity

In [ ]:
# Compare models with and without outliers
# Identify outliers using studentized residuals
from statsmodels.stats.outliers_influence import OLSInfluence

influence = OLSInfluence(model_simple)
studentized_resid = influence.resid_studentized_internal

# Flag outliers (|studentized residual| > 2.5)
outlier_mask = np.abs(studentized_resid) > 2.5

print('\n' + '='*80)
print('ROBUSTNESS CHECK: OUTLIER SENSITIVITY')
print('='*80)
print(f'\nOutliers detected (|studentized residual| > 2.5): {outlier_mask.sum()}/{len(studentized_resid)}')

# Refit model without outliers
df_no_outliers = df_simple[~outlier_mask]
X_no_outliers = sm.add_constant(df_no_outliers[['nps_mean']])
y_no_outliers = df_no_outliers['revenue_total']

model_robust = sm.OLS(y_no_outliers, X_no_outliers).fit()

comparison_robust = pd.DataFrame({
    'Model': ['Original (with outliers)', 'Robust (outliers removed)'],
    'NPS Coefficient': [model_simple.params['nps_mean'], model_robust.params['nps_mean']],
    'R-squared': [model_simple.rsquared, model_robust.rsquared],
    'N': [len(df_simple), len(df_no_outliers)]
})

print('\nModel Comparison:')
print(comparison_robust.to_string(index=False))
print(f'\nCoefficient Change: {abs(model_robust.params["nps_mean"] - model_simple.params["nps_mean"]):.2f}')
print(f'Conclusion: Model is {"stable" if abs(model_robust.params["nps_mean"] - model_simple.params["nps_mean"]) < 100 else "sensitive"} to outliers')

print('\n' + '='*80)
print('REGRESSION ANALYSIS SUMMARY')
print('='*80)

print('\nKEY FINDINGS:')
print('\n1. STRENGTH OF NPS-REVENUE RELATIONSHIP')
print(f'   - Simple model: β₁ = {model_simple.params["nps_mean"]:.2f} (R² = {model_simple.rsquared:.4f})')
print(f'   - Interpretation: 1-point increase in NPS → R${model_simple.params["nps_mean"]:.2f} increase in monthly revenue')
print(f'   - Statistical significance: p = {model_simple.pvalues["nps_mean"]:.6f}')

print(f'\n2. IMPORTANCE OF CLINIC HETEROGENEITY')
print(f'   - Fixed effects model R²: {model_fe.rsquared:.4f} (vs {model_simple.rsquared:.4f} simple)')
print(f'   - Improvement: {(model_fe.rsquared - model_simple.rsquared)*100:.2f} percentage points')
print(f'   - Implication: Clinic-specific factors explain significant revenue variation')

print(f'\n3. TEMPORAL EFFECTS')
print(f'   - Current NPS coefficient: {model_lag.params["nps_mean"]:.2f}')
print(f'   - Lagged NPS coefficient: {model_lag.params["nps_lag1"]:.2f}')
print(f'   - Implication: {"Lagged effect stronger" if abs(model_lag.params["nps_lag1"]) > abs(model_lag.params["nps_mean"]) else "Contemporaneous effect stronger"}')

print(f'\n4. MODELLING RECOMMENDATIONS')
print(f'   - Best-fit model: {best_model_name} (lowest AIC)')
print(f'   - AIC values suggest moderate explanatory power')
print(f'   - Consider: clinic-level random effects, seasonal adjustments, external covariates')

print(f'\n5. PRACTICAL IMPLICATIONS FOR HEALTHCARE OPERATIONS')
print(f'   - NPS improvements SHOULD correlate with revenue growth')
print(f'   - Effect size is clinic-dependent and non-trivial')
print(f'   - Not all revenue variation explained by NPS alone')
print(f'   - Recommend integrated strategy: NPS + operational metrics')

print('\n' + '='*80)

# Save summary
summary_text = f"""
REGRESSION ANALYSIS SUMMARY

Core Finding: NPS has statistically significant relationship with revenue
Effect Size: {model_simple.params['nps_mean']:.2f} R$ per NPS point
Model Fit (R²): {model_simple.rsquared:.4f}
Significance: p = {model_simple.pvalues['nps_mean']:.6f}
Sample Size: {len(df_simple)}

Clinic-level heterogeneity is important (FE R² = {model_fe.rsquared:.4f})
Temporal dynamics present (lagged effects significant)
Robust to outlier removal
"""

with open('../data/processed/reg_final_summary.txt', 'w') as f:
    f.write(summary_text)

print('✓ All regression results saved to processed/ folder')